# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных

• children — количество детей в семье

• days_employed — общий трудовой стаж в днях

• dob_years — возраст клиента в годах

• education — уровень образования клиента

• education_id — идентификатор уровня образования

• family_status — семейное положение

• family_status_id — идентификатор семейного положения

• gender — пол клиента

• income_type — тип занятости

• debt — имел ли задолженность по возврату кредитов

• total_income — ежемесячный доход

• purpose — цель получения кредита

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
# data = pd.read_csv('/Users/Mikalai/Documents/Data Science/Project Jupyter/data.csv')
data = pd.read_csv('/datasets/data.csv')

In [2]:
display(data.head(10))
display(data.describe())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [3]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

Первое знакомство с данными показывает, что таблице 21525 строк и 12 столбцов. 

Типы данных можно было бы не менять, но по заданию требуется вещественный тип данных заменить на целочисленный в столбце total_income. 

В названиях колонок нет нарушения стиля. 

Столбцы days_employed и total_income имеют одинаковое количество пропущенных значений. Возможно здесь есть взаимосвязь. С этими столбцами надо работать в первую очередь.

В других столбцах уже можно заметить некорректные данные. Необходимо исследование данных и по другим столбцам.

### Шаг 2.1 Заполнение пропусков

Проверим, какую долю составляют пропущенные значения в каждом из столбцов с пропусками.

In [4]:
print(f'Процент пропущенные значений = {(len(data) - data.days_employed.count())/len(data):,.2%}')

Процент пропущенные значений = 10.10%


10,1% - это много. Строки с пропусками нельзя просто убрать, так как это повлиляет на результат исследования. Пропуски надо исправлять.

Проверим, это одни и те же строки, где отсутствуют данные по обоим столбцам days_employed и total_income или нет.

In [5]:
data[(data['total_income'].isna() == True) & (data['days_employed'].isna() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Строки совпали, где отсутствуют данные в столбцах days_employed и total_income. Пропусщенные значения могли произойти из-за сбоя при выгрузке или записи данных. 

Для заполнения пропусков сначала проведем исследование данных в столбце days_employed.

In [6]:
display(data[['days_employed']])
display(data[['days_employed']].describe())

,days_employed
0,-8437.673028
1,-4024.803754
2,-5623.422610
3,-4124.747207
4,340266.072047
...,...
21520,-4529.316663
21521,343937.404131
21522,-2113.346888
21523,-3112.481705


,days_employed
count,19351.000000
mean,63046.497661
std,140827.311974
min,-18388.949901
25%,-2747.423625
50%,-1203.369529
75%,-291.095954
max,401755.400475


In [7]:
display([data['days_employed'].value_counts()])

[-8437.673028      1
 -3507.818775      1
  354500.415854    1
 -769.717438       1
 -3963.590317      1
                  ..
 -1099.957609      1
 -209.984794       1
  398099.392433    1
 -1271.038880      1
 -1984.507589      1
 Name: days_employed, Length: 19351, dtype: int64]

In [8]:
data['days_employed'] = data['days_employed'].abs()
display(data[['days_employed']].describe())

,days_employed
count,19351.000000
mean,66914.728907
std,139030.880527
min,24.141633
25%,927.009265
50%,2194.220567
75%,5537.882441
max,401755.400475


In [9]:
days_employed_median = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(days_employed_median)
display(data[['days_employed']].describe())

,days_employed
count,21525.000000
mean,60378.032733
std,133257.558514
min,24.141633
25%,1025.608174
50%,2194.220567
75%,4779.587738
max,401755.400475


Все значения в столбце days_employed заполнены, и отрицательных значений стажа нет.

Проверим заполнение пропусков по столбцу days_employed.

In [10]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

Пропусков данных в столбце days_employed больше нет.

Проведем исследование данных в столбце total_income.

In [11]:
display(data[['total_income']])
display(data[['total_income']].describe())

,total_income
0,253875.639453
1,112080.014102
2,145885.952297
3,267628.550329
4,158616.077870
...,...
21520,224791.862382
21521,155999.806512
21522,89672.561153
21523,244093.050500


,total_income
count,1.935100e+04
mean,1.674223e+05
std,1.029716e+05
min,2.066726e+04
25%,1.030532e+05
50%,1.450179e+05
75%,2.034351e+05
max,2.265604e+06


Проверим предположение, что уровень дохода людей зависит от образования.

In [12]:
income_education = data.groupby('education_id').agg({'total_income':['min','max','mean','median']})
display(income_education)

total_income                                            
                       min           max           mean         median
education_id                                                          
0             32178.213678  2.265604e+06  207142.515219  175340.818855
1             20667.263793  1.726276e+06  153715.643971  136478.643244
2             34466.133539  9.584346e+05  181534.022774  160115.398644
3             25308.586849  4.900673e+05  132155.513626  117137.352825
4             98752.495442  2.684112e+05  174750.155792  157259.898555

Предположение подтвердилолось. Проверим еще есть ли закономерности в появлении пропусков в столбце total_income с другими столбцам

In [13]:
display(data[data['total_income'].isna()].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,2194.220567,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,2194.220567,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,2194.220567,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,2194.220567,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,2194.220567,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,2194.220567,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,2194.220567,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,2194.220567,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,2194.220567,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,2194.220567,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Данные в столбце total_income выглядят корректно. Закономерности в появлении пропусков в столбце total_income с другими столбцами нет, так как везде разные значения. Однако минимальное и максимальное значение месячного дохода отличаются от среднего примерно в 10 раз. Также среднее значение месячного дохода достаточно сильно отличается от медианного. Так уровень дохода зависит от уровня образования, то в данном случае заполним пропуски медианными значениями ежемесячного дохода по каждой группе education. Среднее значение будет некорректно характеризовать данные, когда некоторые значения сильно выделяются среди большинства. 

In [14]:
data['total_income'] = data['total_income'].fillna(data.groupby('education_id')['total_income'].transform('median'))

Проверим заполнение пропусков по столбцам days_employed и total_income.

In [15]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

Пропусков в столбцах нет. Теперь во всех столбцах одинаковое количество данных.

### Шаг 2.2 Проверка данных на аномалии и исправления.

Проверка данных на аномалии в столбце children 

In [16]:
print(data[['children']].value_counts())

children
 0          14149
 1           4818
 2           2055
 3            330
 20            76
-1             47
 4             41
 5              9
dtype: int64


Аномалии в данных столбца children две:
1. Значение 20 детей в 76 случаях. Это маловероятно.Возможно это ошибка при заполнении данных.
2. Значение -1 в 47 случаях. Возможно это ошибка при заполнении данных. Значение -1 могло появиться в результате написания минуса как символа 'тире'.

Проверим, какую долю составляют аномалии в данных столбца children.

In [17]:
print(f'Процент аномальных значений = {(len(data[data.children == 20]) + len(data[data.children == -1]))/len(data):,.2%}')

Процент аномальных значений = 0.57%


Аномалии в данных столбца children составляют 0.57%. Записи с аномалиями в данных столбца children можно удалить, потому что это не повлияет на статистику.

In [18]:
data = data.loc[data['children'] != 20]
data = data.loc[data['children'] != -1]
print(data[['children']].value_counts())

children
0           14149
1            4818
2            2055
3             330
4              41
5               9
dtype: int64


Строки с некорректными записями в столбце children удалены.

Проверка данных на аномалии в столбце dob_years

In [19]:
print(data[['dob_years']].value_counts())

dob_years
35           614
41           603
40           603
34           597
38           595
42           592
33           577
39           572
31           556
36           553
44           543
29           543
30           536
48           536
37           531
43           510
50           509
32           506
49           505
28           501
45           494
27           490
52           483
56           482
47           480
54           476
46           469
58           461
57           457
53           457
51           446
59           441
55           441
26           406
60           376
25           356
61           353
62           351
63           268
64           263
24           263
23           252
65           194
22           183
66           183
67           167
21           110
0            100
68            99
69            83
70            65
71            58
20            51
72            33
19            14
73             8
74             6
75             1
dtyp

Аномальным является нулевой возраст клиента в столбце dob_years, который записан 100 раз. Заменяем нулевой возраст клиента средним возрастом, и изменяем тип данных на целочисленный.

In [20]:
dob_years_mean = data.loc[data.loc[:, 'dob_years'] != 0]['dob_years'].mean()
data['dob_years'] = data['dob_years'].replace(0, dob_years_mean)
data['dob_years'] = data['dob_years'].astype('int')
print(data[['dob_years']].value_counts())

dob_years
35           614
43           610
41           603
40           603
34           597
38           595
42           592
33           577
39           572
31           556
36           553
29           543
44           543
30           536
48           536
37           531
50           509
32           506
49           505
28           501
45           494
27           490
52           483
56           482
47           480
54           476
46           469
58           461
53           457
57           457
51           446
59           441
55           441
26           406
60           376
25           356
61           353
62           351
63           268
24           263
64           263
23           252
65           194
66           183
22           183
67           167
21           110
68            99
69            83
70            65
71            58
20            51
72            33
19            14
73             8
74             6
75             1
dtype: int64


Избавились от нулевый значений возраста клиента. 

Проверка данных на аномалии в столбце gender

In [21]:
print(data[['gender']].value_counts())

gender
F         14154
M          7247
XNA           1
dtype: int64


Некорректное значение XNA встречается 1 раз. Посмотрим строку со значением XNA.  

In [22]:
display(data.loc[data['gender'] == 'XNA'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


По данной записи невозможно определить пол клиента. Запись можно удалить, потому что это не повлияет на статистику.

In [23]:
data = data.loc[data['gender'] != 'XNA']
print(data[['gender']].value_counts())

gender
F         14154
M          7247
dtype: int64


Строка с некорректной записью в столбце gender удалена.

### Шаг 2.3. Изменение типов данных.

Изменяем вещественный тип данных на целочисленный в столбце total_income и проверяем замену.

In [24]:
data['total_income'] = data['total_income'].astype('int')
display(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21401 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21401 non-null  int64  
 1   days_employed     21401 non-null  float64
 2   dob_years         21401 non-null  int32  
 3   education         21401 non-null  object 
 4   education_id      21401 non-null  int64  
 5   family_status     21401 non-null  object 
 6   family_status_id  21401 non-null  int64  
 7   gender            21401 non-null  object 
 8   income_type       21401 non-null  object 
 9   debt              21401 non-null  int64  
 10  total_income      21401 non-null  int32  
 11  purpose           21401 non-null  object 
dtypes: float64(1), int32(2), int64(4), object(5)
memory usage: 2.0+ MB


None

Тип данных в столбце total_income изменен. Во всех столбцах одинаковое количество заполненных данных. 

### Шаг 2.4. Удаление дубликатов.

Для поиска неявных дубликатов посмотрим на данные в столбцах типа object.

In [25]:
print(data[['education']].value_counts())
print()
print(data[['family_status']].value_counts())
print()
print(data[['income_type']].value_counts())
print()
print(data[['purpose']].value_counts())

education          
среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      664
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
dtype: int64

family_status        
женат / замужем          12302
гражданский брак          4159
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
dtype: int64

income_type    
сотрудник          11050
компаньон           5053
пенсионер           3839
госслужащий         1453
безработный            2
предприниматель        2
в декрете              1
студент                1
dtype: int64

purpose                               
свадьба                                   796
на проведение свадьбы    

В столбцах education и family_status при написании слов был использован верхний и нижний регистры. Возможно данные вносят разные люди. Необходимо привести данные к нижнему регистру.

In [26]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()

Определим общее количество дубликатов

In [27]:
data.duplicated().sum()

71

Посмотрим подробно 

In [28]:
display(data.loc[data.duplicated()].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2194.220567,41,среднее,1,женат / замужем,0,F,сотрудник,0,136478,покупка жилья для семьи
3290,0,2194.220567,58,среднее,1,гражданский брак,1,F,пенсионер,0,136478,сыграть свадьбу
4182,1,2194.220567,34,высшее,0,гражданский брак,1,F,сотрудник,0,175340,свадьба
4851,0,2194.220567,60,среднее,1,гражданский брак,1,F,пенсионер,0,136478,свадьба
5557,0,2194.220567,58,среднее,1,гражданский брак,1,F,пенсионер,0,136478,сыграть свадьбу
6312,0,2194.220567,30,среднее,1,женат / замужем,0,M,сотрудник,0,136478,строительство жилой недвижимости
7808,0,2194.220567,57,среднее,1,гражданский брак,1,F,пенсионер,0,136478,на проведение свадьбы
7921,0,2194.220567,64,высшее,0,гражданский брак,1,F,пенсионер,0,175340,на проведение свадьбы
7938,0,2194.220567,71,среднее,1,гражданский брак,1,F,пенсионер,0,136478,на проведение свадьбы
8583,0,2194.220567,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,175340,дополнительное образование


В таблице нет явных строк-дубликатов. Есть только одинаковые данные в столбцах days_employed и income_type, где заполняли пропуски. Поэтому удалять строки-дубликаты не будем.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим новый датафрейм со столбцами education_id и education.

In [29]:
education_dictionary = data[['education_id', 'education']].drop_duplicates().reset_index(drop=True)
display(education_dictionary)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Создадим новый датафрейм со столбцами family_status_id и family_status.

In [30]:
family_dictionary = data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)
display(family_dictionary)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Удаляем из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id. 

In [31]:
data = data.drop(['education', 'family_status'], axis = 1)
display(data.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создадим в таблице столбец total_income_category с категориями:

-	0–30000 — 'E';

-	30001–50000 — 'D';

-	50001–200000 — 'C';

-	200001–1000000 — 'B';

-	1000001 и выше — 'A'.

In [32]:
total_income_data = {'total_income_cat': ['E', 'D', 'C', 'B', 'A'],
                 'total_income_name': ['низкий', 'ниже среднего', 'средний', 
                                       'выше среднего', 'высокий']}
total_income_columns = ['total_income_cat', 'total_income_name']
total_income_dictionary = pd.DataFrame(data = total_income_data, columns = total_income_columns)
display(total_income_dictionary)

,total_income_cat,total_income_name
0,E,низкий
1,D,ниже среднего
2,C,средний
3,B,выше среднего
4,A,высокий


In [33]:
def set_total_income_cat(income):
    if income <= 30000:
        return 'E'
    elif income <= 50000:
        return 'D'
    elif income <= 200000:
        return 'C'
    elif income <= 1000000:
        return 'B'
    else:
        return 'A'

data['total_income_cat'] = data['total_income'].apply(set_total_income_cat)
display(data.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_cat
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

Посмотрим на список уникальных целей получения кредита в столбце purpose

In [34]:
print(data[['purpose']].value_counts())

purpose                               
свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
жилье                                     642
покупка жилья                             641
покупка жилья для семьи                   640
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          622
строительство недвижимости                620
покупка своего жилья                      619
покупка недвижимости                      618
ремонт жилью                              609
покупка жилой недвижимости                603
на покупку своего автомобиля             

В списке содержатся похожие друг на друга цели, но выраженные разными словами. 

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

- 'операции с автомобилем'

- 'операции с недвижимостью'

- 'проведение свадьбы'

- 'получение образования'.

In [35]:
def group_purpose_by_cat(row):
    purpose_group = row['purpose']
    if 'недвиж' in purpose_group:
        return 'операции с недвижимостью'
    if 'жил' in purpose_group:
        return 'операции с недвижимостью'
    elif 'авто' in purpose_group:
        return 'операции с автомобилем'
    elif 'образ' in purpose_group:
        return 'получение образования'
    elif 'свад' in purpose_group:
        return 'проведение свадьбы'
    else:
        return purpose_group

In [36]:
data['purpose_category'] = data.apply(group_purpose_by_cat, axis = 1)
display(data.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_cat,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


Предобработка данных выполнена. Можно приступать к исследованию данных.

### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [37]:
debt_from_children = pd.DataFrame()
debt_from_children['count_credit'] = data.groupby('children')['debt'].count()
debt_from_children['sum_debt'] = data.groupby('children')['debt'].sum()
debt_from_children['ratio'] = debt_from_children['sum_debt'] / debt_from_children['count_credit'] 
debt_from_children.sort_values('ratio', ascending = False)

,count_credit,sum_debt,ratio
children,,,
4,41,4,0.097561
2,2055,194,0.094404
1,4818,444,0.092154
3,330,27,0.081818
0,14148,1063,0.075134
5,9,0,0.000000


##### Вывод 1:

Зависимость между количеством детей и возвратом кредита в срок прослеживается. С увеличением количества детей у кредитополучателей  растет и доля просроченных задолженностей по кредитам. Однако клиенты банка с 3 детьми чаще платят в срок по кредитам, чем люди с 1 ребенком и 2 детьми. Результаты исследования также показывают, что бездетные  реже имеют задолженности по кредитам, чем люди с детьми.

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [38]:
debt_from_family_status = pd.DataFrame()
debt_from_family_status['count_credit'] = data.groupby('family_status_id')['debt'].count()
debt_from_family_status['sum_debt'] = data.groupby('family_status_id')['debt'].sum()
debt_from_family_status['ratio'] = debt_from_family_status['sum_debt'] / debt_from_family_status['count_credit'] 
debt_from_family_status.sort_values('ratio', ascending = False)

,count_credit,sum_debt,ratio
family_status_id,,,
4,2799,273,0.097535
1,4159,385,0.092570
0,12302,927,0.075354
3,1189,84,0.070648
2,952,63,0.066176


##### Вывод 2:

Зависимость между семейным положением и возвратом кредита в срок прослеживается. Клиенты банка не состоящие в браке и состоящие в гражданском браке имеют более высокий процент невозвратов кредитов в срок, чем клиенты банка, состоящих в браке. А те кто овдовели и в разводе чаще платят в срок, чем люди состоящие в браке.

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [39]:
debt_from_total_income = pd.DataFrame()
debt_from_total_income['count_credit'] = data.groupby('total_income_cat')['debt'].count()
debt_from_total_income['sum_debt'] = data.groupby('total_income_cat')['debt'].sum()
debt_from_total_income['ratio'] = debt_from_total_income['sum_debt'] / debt_from_total_income['count_credit'] 
debt_from_total_income.sort_values('ratio', ascending = False)

,count_credit,sum_debt,ratio
total_income_cat,,,
E,22,2,0.090909
C,15993,1353,0.084600
A,25,2,0.080000
B,5012,354,0.070630
D,349,21,0.060172


##### Вывод 3:

Зависимости между уровнем дохода и возвратом кредита не прослеживается.

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

In [40]:
debt_from_purpose_category = pd.DataFrame()
debt_from_purpose_category['count_credit'] = data.groupby('purpose_category')['debt'].count()
debt_from_purpose_category['sum_debt'] = data.groupby('purpose_category')['debt'].sum()
debt_from_purpose_category['ratio'] = debt_from_purpose_category['sum_debt'] / debt_from_purpose_category['count_credit'] 
debt_from_purpose_category.sort_values('ratio', ascending = False)

,count_credit,sum_debt,ratio
purpose_category,,,
операции с автомобилем,4288,400,0.093284
получение образования,3997,369,0.092319
проведение свадьбы,2337,183,0.078306
операции с недвижимостью,10779,780,0.072363


##### Вывод 4:

Чаще платят по кредитам в срок клиенты банка, которые получали кредиты с целью "операции с недвижимостью" и "проведение свадьбы". А вот чаще имеют задолженности те кому был выдан кредит на "операции с автомобилем" и "получение образования". 

## Общий вывод:

1. В работе были проанализированы данные о статистике платежеспособности клиентов банка. Первое знакомство с данными показло, что необходимо провести сначала предобработку данных.

2. В ходе выполнения проекта проведена предобработка данных.

2.1 Обнаружены пропущенные данные в одинаковых записях столбцов days_employed и total_income.  Пропущенные значения могли произойти из-за сбоя при выгрузке или записи данных. Прощенные значения составляют 10,1% от всех данных, поэтому их надо было заполнить.

Перед заполнением пропущенных данные в столбце days_employed отрицательные значения трудового стажа замены на абсолютные.
Пропущенных данные в столбце days_employed заполнены медианным значением по столбцу так как медиана не подвержена имеющимся выбросам значений трудового стажа.

Данные в столбце total_income выглядят корректно, но при этом выяснили, что минимальное и максимальное значение месячного дохода отличаются от среднего примерно в 10 раз, а также среднее значение месячного дохода достаточно сильно отличается от медианного. При этом обнаружено, уровень дохода зависит от уровня образования, то в данном случае пропуски заполнены медианными значениями ежемесячного дохода по каждой группе education.

2.2 В работе далее  проверены данные на аномалии и выполнены исправления некорректных данных.

Аномалии в данных столбца children были две:
- значение 20 детей в 76 случаях. Это маловероятно.Возможно это ошибка при заполнении данных.

- значение -1 в 47 случаях. Возможно это ошибка при заполнении данных. Значение -1 могло появиться в результате написания минуса как символа 'тире'.

Доля некорректных данных в столбце children всего 0.57% от всех данных, поэтому строки с некорректными записями в столбце children удалены.

В столбце dob_years аномальным является нулевой возраст клиента. Выбросов данных в возрасте клиентов банка нет, поэтому заменили нулевой возраст клиента средним возрастом.

Некорректное значение XNA в столбце gender встречается 1 раз, поэтому строка удалена.

2.3 Выполнена замена вещественного типа данных в столбце total_income на целочисленный.

2.4 Обнаружены неявные дубликаты в столбцах education и family_status, где при написании слов был использован верхний и нижний регистры. Возможно данные вносили разные люди. Устранение неявных дубликатов выполено с помощью приведения данных к нижнему регистру.

2.5. Выполнена декомпозиция исходного датафрейма и сформированы два новых датафрейма со столбцами:
- education_id и education — первый;
- family_status_id и family_status — второй.

Удалены из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id.  Данные преобразования были сделаны для сокращения места, занимаемого основной таблицей, а также для наглядности и удобства работы.

2.6 Выполнена категоризация дохода.

На основании диапазонов, указанных ниже, создан столбец total_income_category с категориями:
-	0–30000 — 'E';
-	30001–50000 — 'D';
-	50001–200000 — 'C';
-	200001–1000000 — 'B';
-	1000001 и выше — 'A'.

Данные преобразования были сделаны для наглядности и удобства работы.

2.7 Выполнена категоризация целей кредита.

В списке целей кредита purpose содержались похожие друг на друга цели, но выраженные разными словами.

На основании данных из столбца purpose сформирован новый столбец purpose_category, в который вошли следующие категории:
- 'операции с автомобилем'
- 'операции с недвижимостью'
- 'проведение свадьбы'
- 'получение образования'.

Данные преобразования были сделаны для наглядности и удобства работы.

3. На основании обработанных данных выполнены исследования данных и получены следующие результаты.

3.1 Исследование есть ли зависимость между количеством детей и возвратом кредита в срок?

По результатам исследования такая зависимость прослеживается. Чем больше детей у кредитополучателей, тем больше и задолженностей по своевременной оплате кредитов. Бездетные клиенты реже имеют задолженности по своевременной оплате кредитов, чем клиенты с детьми.

3.2 Исследование есть ли зависимость между семейным положением и возвратом кредита в срок?

По результатам исследования такая зависимость прослеживается. Клиенты банка, бывший в браке и состоящие в браке чаще платят в срок по кредитам чем те, кто не бывал в браке и состоящие в гражданском браке. Причем овдовевшие и разведенные платят в срок чаще, чем люди в браке.

3.3 Исследование есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Зависимости между уровнем дохода и возвратом кредита не прослеживается.

3.4 Исследование как разные цели кредита влияют на его возврат в срок?

Чаще платят по кредитам в срок клиенты банка, которые получали кредиты с целью "операции с недвижимостью" и "проведение свадьбы". А вот чаще имеют задолженности те кому был выдан кредит на "операции с автомобилем" и "получение образования".